In [1]:
# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from env_vars import sentinel_username,sentinel_password
import glob
import pandas as pd
import os
pd.set_option('display.max_colwidth', -1)

/Users/user/.local/share/virtualenvs/data_collection-6QAt_VD6/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [2]:
api = SentinelAPI(sentinel_username, sentinel_password, "https://scihub.copernicus.eu/apihub/")


# download single scene by known product id
# api.download("3f09e58e-b893-46fa-ad05-6bf1f5b4b257")

# search by polygon, time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson('./data/Geometry/congo_basin_boundary/congo_basin_boundary_custom_v3.geojson'))
products = api.query(footprint,
                     date=('20150601', date(2015, 12, 31)),
                     area_relation = "IsWithin",
                     raw='1C',
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 10))


# # # convert to Pandas DataFrame
products_df = api.to_dataframe(products)

FileNotFoundError: [Errno 2] No such file or directory: './data/Geometry/congo_basin_boundary/congo_basin_boundary_custom_v3.geojson'

        Additional query text that will be appended to the query.
    area_relation : {'Intersects', 'Contains', 'IsWithin'}, optional
        What relation to use for testing the AOI. Case insensitive.
    
            - Intersects: true if the AOI and the footprint intersect (default)
            - Contains: true if the AOI is inside the footprint
            - IsWithin: true if the footprint is inside the AOI

In [174]:
products_df.shape

(937, 35)

In [175]:
products_df.columns

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary',
       'datatakesensingstart', 'ingestiondate', 'beginposition', 'endposition',
       'orbitnumber', 'relativeorbitnumber', 'cloudcoverpercentage',
       'filename', 'gmlfootprint', 'format', 'identifier',
       'instrumentshortname', 'sensoroperationalmode', 'instrumentname',
       'footprint', 's2datatakeid', 'platformidentifier', 'orbitdirection',
       'platformserialidentifier', 'processingbaseline', 'processinglevel',
       'producttype', 'platformname', 'size', 'tileid', 'hv_order_tileid',
       'uuid', 'level1cpdiidentifier', 'granuleidentifier',
       'datastripidentifier'],
      dtype='object')

In [176]:
products_df_2 = products_df.drop_duplicates(subset='tileid')

In [177]:
len(products_df_2)

144

In [178]:
online = []
for product_id in products_df_2.index:
    if api.is_online(product_id):
#         print('Product {} is online. Adding to list.'.format(product_id))
        online.append(product_id)
    else:
        continue
print("total online products:",len(online))

total online products: 0


In [163]:
products_df_3 = products_df_2.loc[online]

In [164]:
len(products_df_3)

5

# Run if download was partial within the same year 

In [144]:
downloaded_files = [s.split(".")[0] for s in os.listdir("./data/Sentinel_Data/2015")]

In [145]:
products_df_4 = products_df_3[~products_df_3["title"].isin(downloaded_files)]
products_df_4.shape

(0, 35)

# Converting to OrderedDict in Prep For Bulk Download 

In [146]:
products_df_4["product_id"] = products_df_4.index

In [147]:
from collections import OrderedDict

# Get the unordered dictionary
unordered_products_dict = products_df_4.set_index('product_id').T.to_dict()

 # Then order it
ordered_dict = OrderedDict((k,unordered_products_dict.get(k)) for k in products_df_4.product_id)

In [148]:
products_df_4.head(1)

,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,orbitnumber,...,size,tileid,hv_order_tileid,filename,identifier,uuid,level1cpdiidentifier,granuleidentifier,datastripidentifier,product_id


In [149]:
ordered_dict['55797ce2-d2a1-4e15-826c-75a4f421a21a']

KeyError: '55797ce2-d2a1-4e15-826c-75a4f421a21a'

In [132]:
products['55797ce2-d2a1-4e15-826c-75a4f421a21a']

{'title': 'S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_20150708T082158',
 'link': "https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/$value",
 'link_alternative': "https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/",
 'link_icon': "https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/Products('Quicklook')/$value",
 'summary': 'Date: 2015-07-08T08:10:16.027Z, Instrument: MSI, Mode: , Satellite: Sentinel-2, Size: 152.40 MB',
 'datatakesensingstart': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
 'beginposition': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
 'endposition': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
 'ingestiondate': datetime.datetime(2019, 2, 4, 20, 23, 28, 140000),
 'orbitnumber': 218,
 'relativeorbitnumber': 78,
 'cloudcoverpercentage': 1.3745,
 'gmlfootprint': '<gml:Polygon srsName="http://www.opengis.net/gml/srs/epsg.xml#4326" 

In [ ]:
# download all results from the search
api.download_all(ordered_dict,directory_path="./data/Sentinel_Data/2015/")

Downloading:   0%|          | 0.00/322M [00:00<?, ?B/s]
Downloading:   1%|          | 3.15M/322M [00:01<02:08, 2.48MB/s]
Downloading:   3%|▎         | 8.39M/322M [00:01<01:12, 4.30MB/s]
Downloading:   4%|▎         | 11.5M/322M [00:02<01:06, 4.65MB/s]
Downloading:   5%|▍         | 14.7M/322M [00:02<01:07, 4.59MB/s]
Downloading:   6%|▌         | 17.8M/322M [00:03<00:51, 5.90MB/s]
Downloading:   7%|▋         | 21.0M/322M [00:03<01:08, 4.37MB/s]
Downloading:   7%|▋         | 24.1M/322M [00:04<01:04, 4.60MB/s]
Downloading:   8%|▊         | 27.3M/322M [00:05<00:58, 5.02MB/s]
Downloading:   9%|▉         | 30.4M/322M [00:05<01:03, 4.62MB/s]
Downloading:  10%|█         | 32.5M/322M [00:06<01:10, 4.10MB/s]
Downloading:  11%|█         | 34.6M/322M [00:07<01:29, 3.23MB/s]
Downloading:  11%|█▏        | 36.7M/322M [00:07<01:21, 3.52MB/s]
Downloading:  12%|█▏        | 37.7M/322M [00:08<01:25, 3.33MB/s]
Downloading:  13%|█▎        | 40.9M/322M [00:08<01:05, 4.29MB/s]
Downloading:  14%|█▍        | 45.1

In [57]:
help(api.download_all)

Help on method download_all in module sentinelsat.sentinel:

download_all(products, directory_path='.', max_attempts=10, checksum=True, n_concurrent_dl=2, lta_retry_delay=600) method of sentinelsat.sentinel.SentinelAPI instance
    Download a list of products.
    
    Takes a list of product IDs as input. This means that the return value of query() can be
    passed directly to this method.
    
    File names on the server are used for the downloaded files, e.g.
    "S1A_EW_GRDH_1SDH_20141003T003840_20141003T003920_002658_002F54_4DD1.zip".
    
    In case of interruptions or other exceptions, downloading will restart from where it left
    off. Downloading is attempted at most max_attempts times to avoid getting stuck with
    unrecoverable errors.
    
    Parameters
    ----------
    products : list
        List of product IDs
    directory_path : string
        Directory where the downloaded files will be downloaded
    max_attempts : int, optional
        Number of allowed retr